In [6]:

import csv


def get_data(path):
    data=[]

    with open(path) as fd:
        rd=csv.reader(fd, delimiter="\t", quotechar='"')
        for line in rd:
            li=line[-1]
            data.append(li)
                        
    return data

d = get_data("./eng/nature/CoLA/dev.tsv")

In [7]:
d[0]

'The sailors rode the breeze clear of the rocks.'

In [17]:
def get_QNLI_data(i):
        data=[]
        with open("./eng/nature/QQP/" + i + ".tsv") as fd:
            rd=csv.reader(fd, delimiter="\t", quotechar='"')
            for i, line in enumerate(rd):
                if i == 0:
                    continue
                data.append(line[-2])
                data.append(line[-3])         
        return data

d = get_QNLI_data("dev")

In [18]:
d[0]

'Why are hispanics so beautiful?'

In [1]:
from asyncore import read
import re
import os
import json
from typing import Dict, List
import itertools

from tqdm import tqdm
from transformers import (
    AutoTokenizer,
    MBart50TokenizerFast,
)
import jieba

import sys
sys.path.append("/remote-home/xtzhang/playground/tmp/codedetect/")

from utils.io import read_csv

In [2]:
nature_dir = "./data/eng/nature"

code_dir = "./data/eng/code"


def load_json(data_path):
    
    file = open(data_path, 'r', encoding='utf-8')

    data = []

    for line in file.readlines():
        tmp = json.loads(line)
        data.extend( tmp["original_string"].split("\n") )   

    return data

In [4]:
head="/remote-home/xtzhang/playground/tmp/codedetect/data/eng/code"
dirs = os.listdir(head + "/c")
all_c = []
for d in dirs:
    tmp = read_csv(head + "/c/" + d)
    if tmp:
        all_c.extend(tmp)

'utf-8' codec can't decode byte 0x93 in position 35: invalid start byte
'utf-8' codec can't decode byte 0xd7 in position 400: invalid continuation byte
'utf-8' codec can't decode byte 0x85 in position 929: invalid start byte
'utf-8' codec can't decode byte 0x97 in position 55: invalid start byte
'utf-8' codec can't decode byte 0xed in position 17: invalid continuation byte


In [14]:
def read_code(head="/remote-home/xtzhang/playground/tmp/codedetect/data/eng/code"):
    #dirs = os.listdir( nature_dir )
    types = [ "train", "valid", "test" ]
    program_names = [ "python", "go", "php", "java", "javascript", ]
    
    code_data = { i:[] for i in types }
    trunc_table = {"train":12000, "valid":1500, "test":1500}

    for program_name in program_names:
        for i in types:
            path = "_".join([program_name, i, "0"]) + '.jsonl'
            tmp = load_json( head + "/" +path )
            #print(len(tmp))
            
            code_data[i].extend(tmp[:trunc_table[i]])
    
    dirs = os.listdir(head + "/c")
    all_c = []
    for d in dirs:
        tmp = read_csv(head + "/c/" + d)
        if tmp:
            all_c.extend(tmp)

    import random
    random.shuffle(all_c)
    
    code_data["train"].extend(all_c[:12000])
    code_data["valid"].extend(all_c[12000:13500]) 
    code_data["test"].extend(all_c[13500:])
    
    return code_data
    

In [15]:
code_data = read_code()

'utf-8' codec can't decode byte 0x93 in position 35: invalid start byte
'utf-8' codec can't decode byte 0xd7 in position 400: invalid continuation byte
'utf-8' codec can't decode byte 0x85 in position 929: invalid start byte
'utf-8' codec can't decode byte 0x97 in position 55: invalid start byte
'utf-8' codec can't decode byte 0xed in position 17: invalid continuation byte


In [16]:
for values in code_data.values():
    print(len(values))

72000
9000
9721


In [27]:
def read_nature(head="/remote-home/xtzhang/playground/tmp/codedetect/data/eng/nature"):
    #dirs = os.listdir( nature_dir )
    types = [ "train", "dev", "test" ]
    dataset_names = [ "CoLA", "QNLI", "QQP", "STS-B", "WNLI", ]
    
    nature_data = { i:[] for i in types }
    trunc_table = {"train":12000, "dev":1500, "test":1500}

    for name in dataset_names:
        for i in types:
            path = name + "/" + i + ".tsv"
            tmp = read_csv( head + "/" +path )
            #print(len(tmp))
            
            nature_data[i].extend(tmp[:trunc_table[i]])
    

    return nature_data
    

In [28]:
nature_data = read_nature()

In [29]:
for values in nature_data.values():
    print(len(values))

38937
5615
5591
